In [5]:
import markdown
from markdown.preprocessors import Preprocessor
from markdown.inlinepatterns import InlineProcessor
from markdown.extensions import Extension
import xml.etree.ElementTree as etree
import matplotlib.pyplot as plt
import io
import base64
import re
from jinja2 import Template
from weasyprint import HTML, CSS
from pygments.formatters import HtmlFormatter

# --- 0. 配置 Matplotlib 字体 (让公式更协调) ---
# 使用 STIX Sans 字体，这是一种无衬线数学字体，和网页正文更搭
# 如果报错，可以删掉这一行，回退到默认字体
plt.rcParams['mathtext.fontset'] = 'stixsans' 

# --- 1. 核心渲染函数 ---
def latex_to_base64(latex_str, fontsize=14, color='#374151'):
    """
    渲染 LaTeX 为 SVG。
    """
    try:
        # 画布尺寸设得很小，依靠 bbox_inches='tight' 自动撑开
        fig = plt.figure(figsize=(0.01, 0.01))
        # 渲染文本
        fig.text(0, 0, f"${latex_str}$", fontsize=fontsize, color=color,
                 usetex=False, verticalalignment='center')
        
        output = io.BytesIO()
        # 关键：pad_inches=0.02 减少留白，transparent=True 透明背景
        fig.savefig(output, format='svg', bbox_inches='tight', pad_inches=0.02, transparent=True)
        plt.close(fig)
        
        return base64.b64encode(output.getvalue()).decode('utf-8')
    except Exception as e:
        print(f"渲染失败: {latex_str[:20]}... {e}")
        return None

# --- 2. Markdown 扩展部分 (逻辑保持不变) ---
class MathBlockPreprocessor(Preprocessor):
    def run(self, lines):
        text = "\n".join(lines)
        pattern = re.compile(r'\$\$(.*?)\$\$', re.DOTALL)
        
        def replace_func(match):
            latex = match.group(1).strip()
            # 块级公式生成得稍微大一点点 (fontsize=16)
            img_b64 = latex_to_base64(latex, fontsize=16) 
            if img_b64:
                return f'\n<div class="math-block"><img src="data:image/svg+xml;base64,{img_b64}" /></div>\n'
            return match.group(0)

        new_text = pattern.sub(replace_func, text)
        return new_text.split("\n")

class MathInlineProcessor(InlineProcessor):
    def handleMatch(self, m, data):
        latex = m.group(1)
        # 行内公式使用标准字号 (fontsize=14)
        img_b64 = latex_to_base64(latex, fontsize=14)
        if img_b64:
            el = etree.Element("img")
            el.set("src", f"data:image/svg+xml;base64,{img_b64}")
            el.set("class", "math-inline")
            return el, m.start(0), m.end(0)
        return None, None, None

class MatplotlibMathExtension(Extension):
    def extendMarkdown(self, md):
        md.preprocessors.register(MathBlockPreprocessor(md), 'math_block_pre', 50)
        md.inlinePatterns.register(MathInlineProcessor(r'(?<!\$)\$([^\$]+)\$(?!\$)', md), 'math_inline', 175)

# --- 3. 主渲染器 (CSS 重点修改) ---
class ChatRenderer:
    def _get_syntax_highlighting_css(self):
        return HtmlFormatter(style='friendly').get_style_defs('.codehilite')

    def _generate_html(self, messages):
        pygments_css = self._get_syntax_highlighting_css()

        css = f"""
        <style>
            @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600&family=JetBrains+Mono&display=swap');
            @page {{ size: A4; margin: 0; }}
            
            body {{
                font-family: 'Inter', -apple-system, system-ui, sans-serif;
                background-color: #f3f4f6; 
                padding: 40px 0; 
                margin: 0;
                color: #1f2937;
            }}
            .chat-container {{
                width: 650px; margin: 0 auto; background: white;
                border-radius: 12px; padding: 50px;
                border: 1px solid #e5e7eb;
                box-shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.1);
            }}
            .message {{ margin-bottom: 30px; display: flex; flex-direction: column; page-break-inside: avoid; }}
            .role-label {{ font-size: 12px; font-weight: 700; margin-bottom: 8px; text-transform: uppercase; color: #6b7280; letter-spacing: 0.05em; }}
            
            /* 气泡样式优化 */
            .bubble {{ 
                width: 100%; box-sizing: border-box; 
                padding: 16px 24px; 
                border-radius: 8px; 
                font-size: 15px; 
                line-height: 1.75; /* 增加行高，让公式不挤 */
            }}
            
            .user .role-label {{ color: #2563eb; }}
            .user .bubble {{ background-color: #eff6ff; border: 1px solid #bfdbfe; color: #1e3a8a; }}
            
            .assistant .role-label {{ color: #4b5563; }}
            .assistant .bubble {{ background-color: #f9fafb; border: 1px solid #e5e7eb; color: #374151; }}

            /* 代码块 */
            {pygments_css}
            .codehilite {{ background: white !important; border: 1px solid #e5e7eb; border-radius: 6px; padding: 12px; margin: 12px 0; }}
            pre {{ margin: 0; white-space: pre-wrap; font-family: 'JetBrains Mono', monospace; font-size: 13px; }}

            /* --- 重点：修复公式大小的 CSS --- */
            
            /* 1. 行内公式 */
            img.math-inline {{ 
                height: 1.35em; /* 强制高度为字体的1.35倍 */
                width: auto;    /* 宽度自适应 */
                vertical-align: -0.35em; /* 向下偏移，对齐文字基线 */
                margin: 0 2px; /* 左右留一点点空隙 */
            }}

            /* 2. 块级公式 */
            .math-block {{ 
                display: flex; 
                justify-content: center; 
                margin: 16px 0;
                width: 100%;
            }}
            
            .math-block img {{ 
                height: auto; 
                max-width: 100%; /* 防止撑爆容器 */
                /* 如果块级公式还是太大，可以限制最大高度，例如 max-height: 3em; */
            }}

        </style>
        """

        html_template = """
        <!DOCTYPE html>
        <html>
        <head><meta charset="UTF-8">{{ css }}</head>
        <body>
            <div class="chat-container">
                {% for msg in messages %}
                <div class="message {{ msg.role }}">
                    <div class="role-label">{{ msg.role }}</div>
                    <div class="bubble">{{ msg.content_html }}</div>
                </div>
                {% endfor %}
            </div>
        </body>
        </html>
        """

        math_ext = MatplotlibMathExtension()
        processed_msgs = []
        for msg in messages:
            html_content = markdown.markdown(
                msg["content"], 
                extensions=['fenced_code', 'codehilite', math_ext]
            )
            processed_msgs.append({"role": msg["role"], "content_html": html_content})

        template = Template(html_template)
        return template.render(css=css, messages=processed_msgs)

    def save_as_pdf(self, messages, output_path="final_fixed_size.pdf"):
        print("正在生成 PDF (修复尺寸版)...")
        html_content = self._generate_html(messages)
        HTML(string=html_content, base_url='.').write_pdf(output_path)
        print(f"✅ 成功保存: {output_path}")


# --- 测试效果 ---
if __name__ == "__main__":
    # 一个包含 Markdown 标题、列表、数学公式描述和长代码块的复杂数据
    long_data = [
        {
            "role": "user",
            "content": "我正在学习 Deep Learning，你能详细解释一下 **Transformer** 模型中的 Self-Attention 机制吗？\n\n如果可以的话，请给我一个基于 `PyTorch` 的简化代码实现，方便我理解它的矩阵运算过程。",
        },
        {
            "role": "assistant",
            "content": """
没问题！**Self-Attention (自注意力机制)** 是 Transformer 的核心组件，它解决了 RNN 无法并行计算和长距离依赖的问题。

### 1. 数学公式
注意力机制的计算公式如下，这是一个**行内公式**示例：$Attention(Q, K, V) = softmax(\\frac{QK^T}{\\sqrt{d_k}})V$。

其中，$d_k$ 是 Key 向量的维度。除以 $\\sqrt{d_k}$ 是为了防止点积结果过大导致 Softmax 梯度消失。

下面是一个更复杂的**独立公式块**示例：

$$
\\text{MultiHead}(Q, K, V) = \\text{Concat}(head_1, ..., head_h)W^O
$$

其中每个 head 的计算方式为：

$$
head_i = \\text{Attention}(QW_i^Q, KW_i^K, VW_i^V)
$$

### 2. PyTorch 代码实现
下面是一个简化版的 Multi-Head Self-Attention 实现。即便代码较长，现在的布局也能完美展示：

```python
import torch
import torch.nn as nn

class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        # 定义线性层用来生成 Q, K, V
        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        # 获取 batch size
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # 拆分 Heads
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        query = query.reshape(N, query_len, self.heads, self.head_dim)

        # Einsum 矩阵乘法: query * keys
        energy = torch.einsum("nqhd,nkhd->nhqk", [query, keys])

        # 如果有掩码（Mask），将不需要关注的位置设为负无穷
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        # Softmax 归一化
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)

        # 加权求和: attention * values
        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )

        return self.fc_out(out)
```

这个实现涵盖了矩阵拆分、爱因斯坦求和约定 (einsum) 以及 Mask 的处理。 """,
        },
        {
            "role": "user",
            "content": "非常有帮助！代码里的 torch.einsum 用法确实很简洁。既然你提到了 Mask，如果是 Decoder 部分的 Mask，代码会有什么不同吗？",
        },
        {
            "role": "assistant",
            "content": """ 在 Decoder 中，我们需要用到 Look-ahead Mask (前瞻掩码)。

这是为了防止模型在预测当前词时“偷看”到后面的词。我们通常使用一个上三角矩阵来实现：

```python
# 创建一个上三角矩阵掩码
trg_len = 10
trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
    N, 1, trg_len, trg_len
)

# 在 Attention 计算前应用
energy = energy.masked_fill(trg_mask == 0, float("-1e20"))
```

这样，位置 $i$ 的 query 就只能关注到 $0$ 到 $i$ 位置的 key，而无法看到 $i+1$ 之后的信息。
""",
        },
    ]
    renderer = ChatRenderer()
    # 使用新数据生成 PDF
    renderer.save_as_pdf(long_data, "complex_chat_example.pdf")

正在生成 PDF (修复尺寸版)...
✅ 成功保存: complex_chat_example.pdf
